In [3]:
pip install transformers flask torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.7 MB/s eta 0:00:00


In [7]:
!pip uninstall compressed-tensors -y
!pip install compressed-tensors

Found existing installation: compressed-tensors 0.8.1
Uninstalling compressed-tensors-0.8.1:
  Successfully uninstalled compressed-tensors-0.8.1
  Using cached compressed_tensors-0.8.1-py3-none-any.whl.metadata (6.8 kB)
Using cached compressed_tensors-0.8.1-py3-none-any.whl (87 kB)


In [4]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the quantized model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Path to the tasks JSON file
tasks_file = "tasks.json"

# Load tasks from JSON
def load_tasks(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Save a default tasks file if none exists
def create_default_tasks(file_path):
    default_tasks = {
        "tasks": [
            {"description": "Complete the project report", "status": "pending"},
            {"description": "Prepare slides for the meeting", "status": "pending"},
            {"description": "Review the budget for Q4", "status": "completed"}
        ]
    }
    with open(file_path, "w") as file:
        json.dump(default_tasks, file, indent=4)

# Ensure tasks file exists
try:
    tasks = load_tasks(tasks_file)
except FileNotFoundError:
    create_default_tasks(tasks_file)
    tasks = load_tasks(tasks_file)

# Function to filter and format pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Function to generate responses with task context
def generate_response(prompt, tasks):
    pending_tasks_summary = get_pending_tasks(tasks)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking and managing tasks. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)  # Move inputs to GPU
    outputs = model.generate(**inputs, max_length=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
user_input = "Any suggestions for completing the pending tasks?"
response = generate_response(user_input, tasks)
print(response)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a virtual manager. The user relies on you for tracking and managing tasks. Here are the pending tasks:
- Complete the project report
- Prepare slides for the meeting

Now, respond to the user's query: Any suggestions for completing the pending tasks?

1. Prioritize the tasks: Start with the project report as it might require more time and focus. Once that's done, move on to preparing slides for the meeting.

2. Allocate specific time slots: Assign a specific time frame for each task to ensure they are completed within the deadline.

3. Break down tasks: If the project report is large, break it down into smaller, manageable sections to


In [5]:
# Example usage
user_input = "What are my pending tasks"
response = generate_response(user_input, tasks)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a virtual manager. The user relies on you for tracking and managing tasks. Here are the pending tasks:
- Complete the project report
- Prepare slides for the meeting

Now, respond to the user's query: What are my pending tasks?

Your pending tasks are:
1. Complete the project report
2. Prepare slides for the meeting
